# 지도 이미지에서 Text 추출

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.documents import Document

In [5]:
# 1. pdf 로드

map_path = '../data/olympic.pdf'

loader = PDFPlumberLoader(map_path)
docs = loader.load()
print(docs)
print(docs[0].page_content)

[Document(metadata={'source': '../data/olympic.pdf', 'file_path': '../data/olympic.pdf', 'page': 0, 'total_pages': 1, 'CreationDate': "D:20250731103136+09'00'", 'Creator': 'Adobe Illustrator CS6 (Windows)', 'ModDate': "D:20250731103228+09'00'", 'Producer': 'Adobe PDF library 16.03', 'Title': '올림픽공원_종합안내도'}, page_content='\n')]




In [6]:
import fitz
import numpy as np
from PIL import Image
import easyocr
from langchain.docstore.document import Document

In [7]:
def load_pdf_with_easyocr(
    pdf_path: str,
    languages=("ko","en"),   # 한국어+영어
    gpu=True,                # GPU 사용
    dpi=300,                 # OCR 품질용 렌더 DPI
    page_from=None,          # 1부터 시작, None=첫 페이지
    page_to=None             # 포함, None=마지막 페이지
):
    reader = easyocr.Reader(list(languages), gpu=gpu)
    docs = []

    with fitz.open(pdf_path) as pdf:
        total = len(pdf)
        start = 1 if page_from is None else max(1, page_from)
        end   = total if page_to is None else min(total, page_to)

        for p in range(start-1, end):
            page = pdf[p]
            zoom = dpi / 72.0
            mat = fitz.Matrix(zoom, zoom)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)

            arr = np.array(img)
            result = reader.readtext(arr, detail=0, paragraph=True)
            text = "\n".join(result) if result else ""

            docs.append(Document(
                page_content=text,
                metadata={"source": pdf_path, "page": p+1, "dpi": dpi, "ocr_engine": "EasyOCR"}
            ))

    return docs

In [8]:
docs = load_pdf_with_easyocr(map_path, languages=("ko","en"), gpu=True, dpi=300)
docs

[Document(metadata={'source': '../data/olympic.pdf', 'page': 1, 'dpi': 300, 'ocr_engine': 'EasyOCR'}, page_content='SFDILID8 8\n올림픽공원 종합안내 Olympic Park Information\nKSPos\nOYMPIC PARK\n4과거와 현재가 공존하는 심과 사색의 공간 올림픽공원" 1988년 서울올림픽올 성공적으로 치른 긋이다 한성백제시대 중요한 성각 가운데 하나인 \'동존토성 (사적 제297호)올 중심으로 서울올림픽이 열핏던 사이클 역도 편상 수영 체조 테니스 등 6개의 경기장과 소마(SOMA) 미술관, 한성백제박물관, 올림픽파크텔 등 역사와 문화, 예술과 스포츠가 함께 공존하는 유서 깊은 공간이다. 1982년 경기도; 강원도 충청도 등 전국 각지의 수목 94종 3,892주름 기증받아 아름다운 우리의 자연올 담고 있는 올림픽공원은 \'대한민국 수목 박물관\' 으로도 손색이 없는 긋이다 또한 서울올림픽 \'세계현대 미술제 틀 통해 세계 유명 조각가들의 수준 높은 조형작품올 유치하여고 현재 유명 조각품 222점이 상시 전시되어 세계적인 조각공원으로 불리고 있다. 서울올림픽기념국민체육진홍공단은 서울올림픽의 훌륭한 유산이자 대한민국 공원의 자부심인 올림픽공원올 소중히 보존 하는 한편 시민들의 사색의 공간이자 심과 여유의 공간이 되도록 가꾸어 나가고 있다.\n올림픽대교 Olympic Bridge 용 내 어 통 *J\n북]문 NORTH GATEI\n북?문 NORTH GATE2\n뻔콩 \n편의시설\n안내센터\n공원 매점\n주차장\n편의점\n처진개벗나무\n화장실\n카페\n어린이놀이터\n패스트푸드점\n피크나장\n포켓심터\n음식점\n올림픽존 olympic zone 서울올림픽파크텔 Seoul OlympicParkte 습] 넷치[로스되`움 "기IIUJL7I:-거쥐\n의사이팅존 Exciting zone 올림픽수영경기장 OlympicIndoorSwimmingpool 빗치[ #가흙없

In [11]:
print(docs[0].page_content)

SFDILID8 8
올림픽공원 종합안내 Olympic Park Information
KSPos
OYMPIC PARK
4과거와 현재가 공존하는 심과 사색의 공간 올림픽공원" 1988년 서울올림픽올 성공적으로 치른 긋이다 한성백제시대 중요한 성각 가운데 하나인 '동존토성 (사적 제297호)올 중심으로 서울올림픽이 열핏던 사이클 역도 편상 수영 체조 테니스 등 6개의 경기장과 소마(SOMA) 미술관, 한성백제박물관, 올림픽파크텔 등 역사와 문화, 예술과 스포츠가 함께 공존하는 유서 깊은 공간이다. 1982년 경기도; 강원도 충청도 등 전국 각지의 수목 94종 3,892주름 기증받아 아름다운 우리의 자연올 담고 있는 올림픽공원은 '대한민국 수목 박물관' 으로도 손색이 없는 긋이다 또한 서울올림픽 '세계현대 미술제 틀 통해 세계 유명 조각가들의 수준 높은 조형작품올 유치하여고 현재 유명 조각품 222점이 상시 전시되어 세계적인 조각공원으로 불리고 있다. 서울올림픽기념국민체육진홍공단은 서울올림픽의 훌륭한 유산이자 대한민국 공원의 자부심인 올림픽공원올 소중히 보존 하는 한편 시민들의 사색의 공간이자 심과 여유의 공간이 되도록 가꾸어 나가고 있다.
올림픽대교 Olympic Bridge 용 내 어 통 *J
북]문 NORTH GATEI
북?문 NORTH GATE2
뻔콩 
편의시설
안내센터
공원 매점
주차장
편의점
처진개벗나무
화장실
카페
어린이놀이터
패스트푸드점
피크나장
포켓심터
음식점
올림픽존 olympic zone 서울올림픽파크텔 Seoul OlympicParkte 습] 넷치[로스되`움 "기IIUJL7I:-거쥐
의사이팅존 Exciting zone 올림픽수영경기장 OlympicIndoorSwimmingpool 빗치[ #가흙없- | 71))y7K }#히다 KSPO DOME 올림픽체조경기장 KSPO DOME 모+[로 서 쫓.|*t3 | 71JJ느"시j못3움|#t류 한얼광장 Han-Eul Square HAN-EULr 1J | 시)tJL IGt륙
자연생태존 Picnic zone 동존